#**Construindo um agente de inteligência artificial do zero**

*Um guia simples e prático para construir a sua IA autônoma com LangGraph*



<img src='https://github.com/PatriciaLucas/Minicurso_Agentes_CBIC2025/blob/main/figuras/RAG.png?raw=true' alt="drawing" width="800" />




# **Introdução:** O Advento dos Agentes de IA

  No campo da Inteligência Artificial (IA), um agente é uma entidade artificial projetada para perceber seu ambiente por meio de sensores, tomar decisões com base nessas percepções e executar ações de forma autônoma para alcançar seu(s) objetivo(s). A autonomia, nesse contexto, refere-se à capacidade do agente de operar sem intervenção humana direta, selecionando e realizando ações adequadas diante de diversas situações que possam surgir. Com o surgimento e crescimento dos grandes modelos de linguagem (LLMs), a construção desses agentes tornou-se não apenas mais acessível, como também extremamente poderosa para inúmeras aplicações.

  Perante esse novo cenário de demanda crescente de implementações de agentes para diferentes finalidades, esse texto visa conduzir o leitor através de conceitos e ferramentas essenciais para a construções de agentes inteligentes, permitindo que você crie as suas próprias aplicações sofisticadas.

  

# **LangChain e LangGraph:** As Ferramentas de Construção dos Agentes

Esse tópico é designado para detalhar as duas ferramentas mais importantes para a construção de agentes: **LangChain** e **LangGraph**.

LangChain é um framework projetado para a construção de aplicações em Python que são baseadas em LLMs. Na prática, ele oferece ferramentas para simplificar o desenvolvimento de aplicações de conversação mais complexas e interativas.

Um dos principais componentes do LangChain são os Chat Models, que são modelos de linguagem adaptados especificamente para conversação. Eles possuem funcionalidades extras que permitem a integração com ferramentas externas e a capacidade de trabalhar com entradas e saídas de dados estruturadas.

A comunicação com esses modelos é feita através de uma lista de mensagens, que podem ser dos seguintes tipos:


*   **SystemMessage**: Representa as instruções de contexto e as regras gerais que modelo deve seguir. É utilizada para definir o comportamento do modelo, como o estilo da conversa, a linguagem a ser utilizada e quaisquer limitações que ele deva ter.o

*  **HumanMessage**: Representa a entrada do usuário humano. É o que o modelo interpreta como uma pergunta, um comando ou um dado de entrada para processar.

*  **AIMessage**: Representa a resposta gerada pelo modelo de IA.



# **Potencialização dos agentes:** Retrieval Augmented Generation (RAG)
A geração aumentada por generalização é uma das técnicas mais importantes para dar conhecimento externo ao seu agente. O RAG aprimora as respostas da sua LLM, permitindo que ela consulte uma base externa de conhecimento, atualizada, que funciona como uma memória de longo prazo. Isso resolve o paradigma de limitação do conhecimento das LLMs ao seu treinamento, que acaba deixando os dados estáticos e desatualizados.

O processo do RAG é dividido em duas etapas principais: **indexação** e **recuperação**. Na indexação, documentos externos são carregados, divididos em pedaços menores (chunks), transformados em vetores numéricos (embeddings) que capturam seu significado e armazenados em um banco de dados vetorial (vector store). Quando o usuário faz uma pergunta, a etapa de recuperação busca os chunks mais relevantes nesse banco de dados e os insere no prompt junto à pergunta original.

Para aprofundar seus conhecimentos sobre o tema, consulte o tutorial oficial da LangChain sobre RAG: https://python.langchain.com/docs/tutorials/rag.


# Overview do grafo e escolha da LLM

Um grafo de controle é o mapa mental do sistema de agentes. A partir dele é definido como o fluxo de pensamento e as ações do agente devem acontecer a partir do prompt que o usuário envia. Esses grafos são direcionados, possuindo um vértice inicial (start) e um vértice final (end). Se quiser entender melhor sobre grafos, acesse esse [link](https://pt.wikipedia.org/wiki/Teoria_dos_grafos). O grafo a seguir representa o fluxo de raciocínio dos agentes neste projeto.


<img src='https://github.com/PatriciaLucas/Minicurso_Agentes_CBIC2025/blob/main/figuras/grafo_assistente.png?raw=true' alt="drawing" width="300" />

Quando o agente entra em operação, ele parte do nó inicial e segue para o assistente. Nesse ponto, uma LLM previamente definida recebe o prompt e o interpreta.

Em seguida, o fluxo pode passar por dois **nós intermediários**:

1. O primeiro é o *tools*, que representa o conjunto de ferramentas externas que o agente pode acionar para auxiliar na resposta.
2. O segundo é o *moderador*, onde outra instância de LLM verifica se a resposta gerada é adequada ao problema e, se necessário, fornece feedback ao assistente principal.

**Dependendo da entrada**, o assistente pode seguir para um desses nós ou encerrar o fluxo de pensamento diretamente. Esse grafo permite visualizar, em tempo real, como o agente está raciocinando e em qual estágio do processo ele se encontra.

A escolha da LLM é livre, mas no caso do nó assistente, ela precisa oferecer suporte a chamadas de ferramentas (tool calling), para que o agente possa interagir com o nó tools quando necessário. Além disso, o poder da LLM deve estar alinhado à complexidade do problema: **quanto mais exigente a tarefa, mais robusto deve ser o modelo**, como ChatGPT, Gemini ou outros equivalentes.

O processo de inicialização do agente, juntamente com a definição de qual LLM será utilizada, será exibida em código logo abaixo.

In [ ]:
# definindo o modelo de LLM que será utilizado e alguns parâmetros úteis.
model = ChatDeepInfra(
            model= "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",
            temperature=0,       # configura o modelo com amostragem determinística
            max_tokens = 256,    # número máximo de tokens que o modelo pode gerar
            deepinfra_api_token=API_KEY
        )

# a partir do modelo de LLM, inicializamos um agente específico para a tarefa de
# ser o assistente.
agent_assistente = initialize_agent(
    tools, # ferramentas externas que ele terá acesso.
    model,
    agent="zero-shot-react-description",  # usa a técnica ReAct.
    verbose=True
)

# o agente moderador é uma simples instância da LLM escolhida.
agent_moderador = model

Neste projeto, utilizamos a infraestrutura da DeepInfra para acessar as LLMs, mas o processo é muito semelhante ao de outros provedores.

Após inicializar o modelo, o próximo passo é criar o agente assistente, **integrando a LLM às suas tools e ao modelo de comportamento [ReAct](https://arxiv.org/abs/2210.03629)**. Esse modelo combina o raciocínio em cadeia (chain of thought) com a execução de ações, permitindo que o agente pense sobre o problema e use suas ferramentas de forma integrada e inteligente.

Também é criado o agente moderador, como uma simples instância do modelo LLM escolhido. Isso é feito, pois esse agente não necessita de ferramentas externas nem de um padrão de comportamento específico como o ReAct.

#Tools
Como mencionado anteriormente, as tools são ferramentas externas à LLM que o agente pode acessar para buscar informações específicas ou executar tarefas especializadas.

Uma tool pode ser, por exemplo, uma função que conecta o agente a um serviço externo — como uma API de previsão do tempo, a API da NASA para obter dados astronômicos em tempo real, ou até uma função responsável por realizar um cálculo específico.

É um recurso bastante versátil, e neste projeto utilizamos uma única tool, chamada **retrieve_context**, responsável por fazer a recuperação de informações do vector store. Observe a implementação dela logo abaixo.

In [ ]:
# Criação de uma função responsável pelo RAG
def rag(documento):
    global retriever

    # Exemplo com URL
    urls = [documento]

    # Load documentos
    loader = UnstructuredURLLoader(urls=urls)
    docs = loader.load()

    # Split documentos
    text_splitter = RecursiveCharacterTextSplitter(separators = ["\n\n", "\n", ". ", ", ", " ", ""],
                                                   chunk_size=200,
                                                   chunk_overlap=20)
    doc_splits = text_splitter.split_documents(docs)

    # Criação VectorStore
    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        collection_name="docs",
        embedding = DeepInfraEmbeddings(model_id="BAAI/bge-base-en-v1.5", deepinfra_api_token=API_KEY), # Chamada do modelo de embedding
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # Instancia a VectorStore para buscar por k nos documentos internos
    # que estejam relacionados com o tema requerido no prompt

    return retriever


# Criação da Tool responsável pelo RAG
@tool
def retrieve_context(query: str):
    """Pesquise notícias recentes sobre astronomia."""
    global retriever
    results = retriever.invoke(query) # Chama a VectorStore para pesquisar por documentos com base na query
    print(results)
    return "\n".join([doc.page_content for doc in results])

# Adiciona a ferramenta na lista de tools que é fornecida para o agente
tools = [retrieve_context]

Repare que o RAG é o responsável por todo o processo de ingestão e preparação dos dados: ele absorve os documentos, os converte em vetores e os armazena em uma vector store.

A tool atua sobre esse conjunto, **buscando dentro dos documentos as informações relevantes**. Neste caso, buscaremos tópicos sobre astronomia.

Quando o agente opta por acionar essa ferramenta, ele escolhe uma palavra chave para fazer a consulta. Assim, internamente é chamado o método invoke a partir do retriever, uma instância da VectorStore encarregada de recuperar os dados vetorizados.

Dessa forma, sempre que o agente precisa pesquisar notícias recentes sobre astronomia, ele aciona essa tool, terceirizando a busca e garantindo uma resposta mais precisa e contextualizada.

#State
No LangGraph, o State é uma estrutura de dados compartilhada entre todos os nós do grafo.

Ele funciona como uma memória transitória, armazenando o contexto da execução e as informações relevantes. Isso permite que cada nó leia, modifique e repasse esses dados ao longo do fluxo.

O state do nosso grafo foi definido da seguinte forma:

In [ ]:
class State(TypedDict):
    mensagens: List[BaseMessage]
    documento: str
    avaliacao: str
    feedback: str

Repare que, no código, o State é uma classe. No nosso caso, temos os seguintes atributos:

* **mensagens:** reúnem todo o histórico de mensagens, desde os

comandos do sistema até as interações do usuário e as respostas geradas.

* **documento:** representa o conteúdo carregado no RAG.

* **avaliação** indicam se há necessidade de o assistente revisar ou reprocessar sua resposta.

* **feedback** mensagem enviada do agente moderador ao agente assistente, informando sobre a correção ou ajuste que deve ser realizado em resposta a uma avaliação negativa.

# Definição dos nós
No LangGraph, os nós correspondem a funções que implementam a lógica dos agentes. Cada nó recebe o estado atual como entrada, executa um processamento específico e retorna um estado atualizado.

No nosso agente de astronomia será necessário definir 3 nós, sendo eles: nó moderador, nó assistente e nó roteador.  

##Nó Assistente
Este é o nó que utiliza o agente que será especializado em astronomia, ele tem acesso às ferramentas externas definidas anteriormente que perimitirão construir respostas completas e especializadas em astronomia. Assim como os nós do LangGraph ele recebe um estado e após o processsamento retorna um estado atualizado.

In [ ]:
# Nó assitente
def assistente(state: State):
    #Retem informações do documento
    global retriever
    retriever = rag(state["documento"])
    feedback = state.get("feedback", "")

    # Pega a última mensagem humana. O caso 2 é para funcionar no langsmith.
    last_human_message = None
    for msg in reversed(state["messages"]):
        # caso 1: já é HumanMessage
        if isinstance(msg, HumanMessage):
            last_human_message = msg.content
            break
        # caso 2: veio como dict serializado
        if isinstance(msg, dict) and msg.get("type") == "human":
            last_human_message = msg.get("content")
            break

    #Definição da SystemMessage para definição do comportamento do agente assistente
    prompt_system = f"""
    Você é um assistente de astronomia super gentil que se comunica em português.
    """
    #Definição da HumanMessage: configuração do Prompt Base para geração de resposta
    prompt_assistente = f"""
    Responda a Pergunta usando a dica para formular melhor sua resposta.

    Pergunta: {last_human_message}
    Dica: {feedback}

    Responda APENAS em JSON válido no formato:
    {{"resposta": "sua resposta aqui"}}

    """
    #Contrução da lista de mensagens e invocação do agente assistente
    messages = [SystemMessage(content=prompt_system), HumanMessage(content=prompt_assistente)]
    response = agent_assistente.invoke(messages)

    result = json.loads(response['output'])

    resposta = result.get("resposta", "").strip()

    #Atualização do estado com a resposta do agente assistente
    state["messages"] = state["messages"] + [AIMessage(content=resposta)]

    return state


Quando o nó assistente é executado, o mecanismo de busca RAG é inicializado com base no documento atual, o feedback disponível e a última mensagem humana registrada no histórico são recuperados.

Em seguida, um prompt combinando a pergunta do usuário e o feedback fornecido pelo agente moderador é construído. Esse prompt é então enviado ao agente assistente, que retorna uma resposta em formato JSON. O conteúdo gerado é extraído e incorporado ao histórico de mensagens, resultando na atualização do estado da conversa.

## Nó Moderador
Este nó implementa o agente moderador que tem a função de avaliar se a resposta produzida pelo agente assistente é adequada e satisfatória. Se a resposta atender aos critérios esperados, o processo é concluído e o usuário a recebe. Caso contrário, o moderador gera um feedback construtivo, que é incorporado ao prompt do agente assistente (como visto anteriormente) para orientar uma nova formulação da resposta.

In [ ]:
def moderador(state: State):

    # Pega a última mensagem AI. O caso 2 é para funcionar no langsmith.
    last_ai_message = None
    for msg in reversed(state["messages"]):
        # caso 1: já é AIMessage
        if isinstance(msg, AIMessage):
            last_ai_message = msg.content
            break
        # caso 2: veio como dict serializado
        if isinstance(msg, dict) and msg.get("type") == "ai":
            last_ai_message = msg.get("content")
            break

    # Pega a última mensagem humana. O caso 2 é para funcionar no langsmith.
    last_human_message = None
    for msg in reversed(state["messages"]):
        # caso 1: já é HumanMessage
        if isinstance(msg, HumanMessage):
            last_human_message = msg.content
            break
        # caso 2: veio como dict serializado
        if isinstance(msg, dict) and msg.get("type") == "human":
            last_human_message = msg.get("content")
            break

    # Prompt template
    parser = JsonOutputParser()

    # Prompt que determina o comportamento do agente moderador e seu objetivo
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Você é um moderador de RESPOSTAS de um agente que só deve falar sobre o tema astronomia."),
        ("human",
         """Dada a Pergunta e a Resposta, responda se a Resposta está adequada para a Pergunta.
         Responda "sim" se a resposta estiver adequada e "não" se a resposta não estiver adequada.

         Se sua resposta for não, dê um feedback curto para que o agente assistente melhore sua resposta.

         Responda APENAS em JSON válido no formato:
         {{"resposta": "sim", "feedback": "Você não possui feedback."}}
         ou
         {{"resposta": "não", "feedback": "Escreva seu feedback aqui"}}.

         Pergunta: {pergunta}
         Resposta: {resposta}"""
        )
    ])

    # Invocação do agente moderado com tratamento de erro
    chain = prompt | agent_moderador | parser
    try:
        result = chain.invoke({"pergunta": last_human_message, "resposta": last_ai_message})
        resposta = result.get("resposta", "").strip()
        feedback = result.get("feedback", "").strip()

    except Exception as e:
        print("Erro ao interpretar resposta:", e)
        resposta = "sim"
        feedback = "Você não possui feedback."

    print({"avaliacao": resposta,"feedback": feedback})
    # Atualização do estado com a resposta e feedback do agente assistente
    state["avaliacao"] = resposta
    state["feedback"] = feedback

    return state

Quando o nó moderador é executado, a última mensagem enviada pelo assistente e a última pergunta feita pelo usuário são recuperadas. Em seguida, um prompt que instrui o agente a agir como um avaliador é construído, verificando se a resposta do assistente está adequada à pergunta e restrita ao tema de astronomia. O modelo então responde em formato JSON, indicando “sim” ou “não” e, caso necessário, fornece um breve feedback. En seguida, a avaliação e o feedback são atualizados no estado, permitindo que o direcionamento para o fim da conversa ou de volta pata o agente assistente.

##Nó Roteador
Este nó tem como função determinar o próximo passo do fluxo com base na avaliação realizada pelo agente moderador. Esse tipo de nó é comum em arquiteturas desenvolvidas com o LangGraph, nas quais os nós atuam como etapas de decisão ou de processamento dentro do grafo.

In [ ]:
def roteador(state: State):
    """Roteia para o agente_assistente ou finaliza."""
    avaliacao = state.get("avaliacao")
    #Caso a avaliação seja negativa, voltamos ao nó assistente
    if avaliacao == 'não':
            return "refazer"

    return 'fim'


Primeiramente, recupera-se a avaliacao feita pelo agente moderador, que indica se a resposta anterior foi considerada adequada ou não. Caso o valor seja "não", o roteador redireciona o fluxo para "refazer", retornando ao agente assistente para que a resposta seja reformulada com base no feedback fornecido. Se a avaliação for positiva, retorna "fim", encerrando o processo e permitindo que a resposta final seja enviada ao usuário.

#Criação do Grafo
Esse trecho define e organiza todo o fluxo de execução do grafo no LangGraph, estruturando como os nós se conectam entre si e em que ordem serão executados, assim como foi definido e explicado na seção 'Overview do grafo e escolha da LLM'.

In [ ]:
# Definindo o grafo
workflow = StateGraph(State)

# Adicionando os nós
tool_node = ToolNode(tools=tools)
workflow.add_node("assistente", assistente)
workflow.add_node("tools", tool_node)
workflow.add_node("moderador", moderador)

# Conectando os nós
workflow.add_edge(START, "assistente")
workflow.add_conditional_edges("assistente", tools_condition)
workflow.add_edge("tools", "assistente")
workflow.add_edge("assistente", "moderador")
workflow.add_conditional_edges(
    "moderador",
    roteador,
    {
        "refazer": "assistente",
        "fim": END
    }
)


O grafo é conectado iniciando no nó "assistente" (workflow.add_edge(START, "assistente")), que pode seguir para o nó "tools" conforme a condição definida em tools_condition. O nó "tools" retorna ao "assistente", permitindo que o agente use ferramentas antes de prosseguir. Em seguida, o fluxo avança para o "moderador", responsável por avaliar a resposta gerada.

Por fim, a conexão condicional workflow.add_conditional_edges("moderador", roteador, {"refazer": "assistente", "fim": END}) define o comportamento do grafo após a moderação: se o roteador indicar que a resposta precisa ser refeita, o fluxo retorna ao "assistente"; se estiver adequada, o processo é encerrado (END).

#Compilação do Grafo
Esse trecho adiciona memória ao grafo e o prepara para execução. O MemorySaver() cria um checkpointer que salva o estado da conversa, permitindo retomar o fluxo ou manter o histórico entre interações. Em seguida, workflow.compile(checkpointer=checkpointer) compila o grafo com essa funcionalidade, gerando o aplicativo (app) pronto para execução com suporte à memória.

In [ ]:
# Para incluir memória inclua checkpointer no compile.
checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)

#Execução do Agente de Astronomia
Agora, finalmente, podemos executar nosso agente de astronomia e fazer perguntas a ele!


In [ ]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="Olá! Quais as novidades astronômicas de hoje?"),],
        "documento": 'https://www.nasa.gov/news/recently-published/',
        "avaliacao": "",
        "feedback": ""
     },
    config={"configurable": {"api_key": API_KEY, "thread_id": 42}}
)

# Estado final do grafo
print(final_state)

# Saída do print
{'messages': [HumanMessage(content='Olá! Quais as novidades astronômicas de hoje, dia 24/10/2025?', additional_kwargs={}, response_metadata={}),

              AIMessage(content='''Uma das novidades astronômicas de hoje, dia 24/10/2025, é a chuva de meteoros Orionid, que está brilhando intensamente no céu.
              Esse evento é um dos destaques da astronomia para o mês de outubro.''',
              additional_kwargs={}, response_metadata={})],

 'documento': 'https://www.nasa.gov/news/recently-published/',

 'avaliacao': 'sim',

 'feedback': 'Você não possui feedback.',
}


Esse trecho executa o grafo completo. A função app.invoke() inicia o fluxo passando um estado inicial com a mensagem do usuário e o link do documento usado pelo RAG . O parâmetro config define informações adicionais, como a chave da API e o identificador da conversa. Ao final da execução, o resultado, que contém todas as mensagens e dados gerados pelo fluxo, é armazenado em final_state e exibido com print(final_state), mostrando o estado final do grafo após todo o processamento.

Neste passo a passo, vimos que é totalmente possível transformar uma ideia em um agente LLM funcional, capaz de buscar informações, compreendê-las e gerar respostas de forma autônoma. Foi construído um fluxo lógico e integrada uma fonte de conteúdo que permitiu ao agente aprender a revisar o que diz antes de responder. O mais interessante é que esse esqueleto pode ser adaptado para qualquer tema: basta trocar a fonte, adicionar novas “habilidades” e você terá um assistente inteligente moldado ao seu próprio universo!